In [ ]:
import os

In [ ]:
!pip install ../input/efficientnet-pytorch/dist/efficientnet_pytorch-0.7.1.tar

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
from PIL import Image     
from tqdm import tqdm       
import copy 
import torchvision
from torchvision import transforms, models    
from torch import optim        
import torch.nn as nn
from torch.optim import lr_scheduler    
import torch     
import numpy as np
from glob import glob   
import pandas as pd

In [ ]:
transform_test = transforms.Compose([
        transforms.Resize(256),   
        transforms.CenterCrop(224),   
        transforms.ToTensor(),   
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]) 
    ])

In [ ]:
import random
with open('../input/plant-pathology-2021-fgvc8/sample_submission.csv', 'r') as f:   
    csv = f.readlines()[1:]      
    test_csv = csv      

In [ ]:
test = {i.split(',')[1] for i in test_csv}
print({label:idx for idx, label in enumerate(test)}) 

In [ ]:
class torchvision_Dataset(torch.utils.data.Dataset): 
    def __init__(self, data_root, csv, transforms=None):     
        self.data = csv    
        self.image_path = data_root     
        label = {i.split(',')[1] for i in self.data}  
        self.label = {label:idx for idx, label in enumerate(label)}   
        self.transform = transforms
        
    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):                    
        image_name, label_name = self.data[idx].split(',')  
        img = Image.open(os.path.join(self.image_path, image_name))
        if self.transform:
            x = self.transform(img)
        return x, self.label[label_name]

In [ ]:
test_dataset = torchvision_Dataset('../input/plant-pathology-2021-fgvc8/test_images', test_csv, transform_test)

In [ ]:
test_dataloaders = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4) 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")   
print(device)

In [ ]:
model_ft = EfficientNet.from_name('efficientnet-b7', num_classes=12)
model_ft.to(device)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.05, momentum=0.9) 
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
model = model_ft
model.load_state_dict(torch.load('../input/efficientnetb7/efficientnet-b7.pt'))

In [ ]:
model.to(device)

In [ ]:
import csv
img_paths = glob("../input/plant-pathology-2021-fgvc8/test_images/*")
label2idx ={'rust\n': 0, 'powdery_mildew\n': 1, 'complex\n': 2, 'scab frog_eye_leaf_spot complex\n': 3, 'frog_eye_leaf_spot\n': 4, 'scab\n': 5, 'scab frog_eye_leaf_spot\n': 6, 'powdery_mildew complex\n': 7, 'frog_eye_leaf_spot complex\n': 8, 'healthy\n': 9, 'rust complex\n': 10, 'rust frog_eye_leaf_spot\n': 11}
idx2label = {label2idx[label]:label for label in label2idx}
from glob import glob
import csv

submit =[] 

for img_path in img_paths:
    model.eval()
    img = Image.open(img_path)
    img = transform_test(img)
    img = img.unsqueeze(0)
    with torch.no_grad():
        pred = model(img.cuda())
    _, top_one = torch.max(pred, 1)
    img_path = img_path.split('/')[-1]
    
    submit.append([img_path,idx2label[int(top_one)]])
    
submission = pd.DataFrame(submit, columns=["image", "labels"])
submission.to_csv("/kaggle/working/submission.csv", index=False)